In [1]:
from collections import Counter
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch

from datasets import load_dataset
import accelerate
from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    DataCollatorForTokenClassification,
    TrainingArguments,
    Trainer
)

from evaluate import load
import collections

In [2]:
raw_dataset = load_dataset("ai4privacy/pii-masking-200k")

In [3]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['source_text', 'target_text', 'privacy_mask', 'span_labels', 'mbert_text_tokens', 'mbert_bio_labels', 'id', 'language', 'set'],
        num_rows: 209261
    })
})

In [4]:
features = raw_dataset['train'].features
print(features)

{'source_text': Value('string'), 'target_text': Value('string'), 'privacy_mask': List({'value': Value('string'), 'start': Value('int64'), 'end': Value('int64'), 'label': Value('string')}), 'span_labels': Value('string'), 'mbert_text_tokens': List(Value('string')), 'mbert_bio_labels': List(Value('string')), 'id': Value('int64'), 'language': Value('string'), 'set': Value('string')}


In [5]:
train_ds = raw_dataset['train']
train_ds

Dataset({
    features: ['source_text', 'target_text', 'privacy_mask', 'span_labels', 'mbert_text_tokens', 'mbert_bio_labels', 'id', 'language', 'set'],
    num_rows: 209261
})

In [6]:
for i in train_ds[0]:
    print(i)

source_text
target_text
privacy_mask
span_labels
mbert_text_tokens
mbert_bio_labels
id
language
set


In [7]:
train_ds[0]

{'source_text': "A student's assessment was found on device bearing IMEI: 06-184755-866851-3. The document falls under the various topics discussed in our Optimization curriculum. Can you please collect it?",
 'target_text': "A student's assessment was found on device bearing IMEI: [PHONEIMEI]. The document falls under the various topics discussed in our [JOBAREA] curriculum. Can you please collect it?",
 'privacy_mask': [{'value': '06-184755-866851-3',
   'start': 57,
   'end': 75,
   'label': 'PHONEIMEI'},
  {'value': 'Optimization', 'start': 138, 'end': 150, 'label': 'JOBAREA'}],
 'span_labels': '[[0, 57, "O"], [57, 75, "PHONEIMEI"], [75, 138, "O"], [138, 150, "JOBAREA"], [150, 189, "O"]]',
 'mbert_text_tokens': ['A',
  'student',
  "'",
  's',
  'assessment',
  'was',
  'found',
  'on',
  'device',
  'bearing',
  'IM',
  '##E',
  '##I',
  ':',
  '06',
  '-',
  '1847',
  '##55',
  '-',
  '866',
  '##85',
  '##1',
  '-',
  '3',
  '.',
  'The',
  'document',
  'falls',
  'under',
  't

In [8]:
label_set = set()

for labels in train_ds['mbert_bio_labels']:
    label_set.update(labels)

label_list = sorted(label_set)

In [9]:
label2id = {label: idx for idx, label in enumerate(label_list)}
id2label = {idx: label for label, idx in label2id.items()}

In [10]:
def encode_labels(example):
	example['ner_tags'] = [label2id[i] for i in example['mbert_bio_labels']]
	return example
''' example - variable named in reference to each "training example". '''
		
train_ds = train_ds.map(encode_labels)

In [11]:
splits = train_ds.train_test_split(test_size=0.2, seed=42)
train_ds, test_ds = splits['train'], splits['test']

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [13]:
model_name = "bert-base-multilingual-cased"

In [14]:
from transformers import AutoTokenizer
from transformers import AutoModelForTokenClassification
model_name = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(
    model_name,
    num_labels=len(label_list),
    id2label=id2label,
    label2id=label2id
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
model = model.to(device)

In [16]:
sample_text = (
    "Hello, my name is Sarah Johnson. "
    "I live at 42 Maple Street, New York, and my email is sarah.j@gmail.com. "
    "My phone number is 555-1234."
)

In [17]:
encoding = tokenizer(
    sample_text,
    return_tensors='pt',
    truncation=True,
    padding="max_length",
    max_length=512,
)

encoding = encoding.to(device)
model.eval()
with torch.no_grad():
    outputs = model(**encoding)
    
predictions = outputs.logits.argmax(dim=-1).squeeze().tolist()
tokens = tokenizer.convert_ids_to_tokens(encoding["input_ids"].squeeze())
decoded_preds = [id2label[p] for p in predictions]


for token, label in zip(tokens, decoded_preds):
    print(f"{token:15} -> {label}")

[CLS]           -> B-BITCOINADDRESS
Hello           -> I-CREDITCARDNUMBER
,               -> I-PASSWORD
my              -> B-CREDITCARDCVV
name            -> B-COUNTY
is              -> B-IP
Sarah           -> I-PASSWORD
Johnson         -> B-IPV6
.               -> B-COUNTY
I               -> B-COUNTY
live            -> B-COUNTY
at              -> I-EMAIL
42              -> B-USERNAME
Maple           -> B-COUNTY
Street          -> I-ZIPCODE
,               -> I-PASSWORD
New             -> B-COUNTY
York            -> I-ZIPCODE
,               -> I-PASSWORD
and             -> I-PASSWORD
my              -> B-STREET
email           -> I-PASSWORD
is              -> B-IP
sa              -> B-IP
##rah           -> B-MIDDLENAME
.               -> I-PHONENUMBER
j               -> I-NEARBYGPSCOORDINATE
@               -> I-EMAIL
g               -> B-CREDITCARDCVV
##mail          -> I-CREDITCARDNUMBER
.               -> B-CURRENCY
com             -> I-CURRENCYSYMBOL
.               -> I-ZIPCODE
M

Performing badly

In [18]:
def tokenize_batch(batch, tokenizer, max_length=512):
    return tokenizer(
        batch['mbert_text_tokens'],
        truncation=True,
        is_split_into_words=True,
        padding="max_length",
        max_length = max_length
    )

In [19]:
def align_labels_for_tokenized_sample(word_ids, word_labels):
    """Align word-level labels with tokenized subwords."""
    aligned_labels = []
    prev_word = None
    for word_idx in word_ids:
        if word_idx is None:
            # [CLS], [SEP], and padding
            aligned_labels.append(-100)
        else:
            label_id = word_labels[word_idx]
            if word_idx != prev_word:
                aligned_labels.append(label_id)
            else:
                # subsequent subwords are ignored in loss
                aligned_labels.append(-100)
        prev_word = word_idx
    return aligned_labels

In [20]:
def align_labels_batch(tokenized_batch, batch):
    """Align labels for the entire batch after tokenization."""
    labels = []
    labels = []
    for i in range(len(batch["mbert_text_tokens"])):
        word_ids = tokenized_batch.word_ids(batch_index=i)
        word_labels = batch["ner_tags"][i]
        aligned_labels = align_labels_for_tokenized_sample(word_ids, word_labels)
        labels.append(aligned_labels)
    return labels

In [21]:
def tokenize_and_align_labels(batch):
    """Full pipeline for tokenizing and aligning labels."""
    tokenized_batch = tokenize_batch(batch, tokenizer)
    labels = align_labels_batch(tokenized_batch, batch)
    tokenized_batch["labels"] = labels
    return tokenized_batch

In [22]:
tokenized_ds = train_ds.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=train_ds.column_names
)

val_tokenized_ds = test_ds.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=test_ds.column_names
)

Map:   0%|          | 0/167408 [00:00<?, ? examples/s]

Map:   0%|          | 0/41853 [00:00<?, ? examples/s]

In [23]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [24]:
metric = load("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_labels = [
        [id2label[l] for l in label if l != -100] 
        for label in labels
    ]
    true_preds = [
        [id2label[p] for (p, l) in zip(pred, label) if l != -100]
        for pred, label in zip(predictions, labels)
    ]
    results = metric.compute(predictions=true_preds, references=true_labels)
    # Return entity-level metrics
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [25]:
import torch
print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("Torch CUDA version:", torch.version.cuda)
print("GPU count:", torch.cuda.device_count())

Torch version: 2.7.1+cu118
CUDA available: True
Torch CUDA version: 11.8
GPU count: 1


In [26]:
import transformers, accelerate
print(transformers.__version__)
print(accelerate.__version__)

4.57.3
1.12.0


In [27]:
training_args = TrainingArguments(
    output_dir="./pii_bert_model",     
    save_strategy="epoch",           
    learning_rate=3e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_steps=50,
    report_to=[],
    fp16=torch.cuda.is_available()    
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds,
    eval_dataset=val_tokenized_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

C:\Users\mjaye\AppData\Local\Temp\ipykernel_17264\3051544704.py:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [28]:
print(accelerate.__version__)

1.12.0


In [29]:
trainer.train()

Step,Training Loss
50,1.927300
100,0.770100
150,0.546500
200,0.409800
250,0.305300
300,0.265800
350,0.212400
400,0.190800
450,0.174700
500,0.171100


TrainOutput(global_step=26160, training_loss=0.05600325330829857, metrics={'train_runtime': 37498.981, 'train_samples_per_second': 22.322, 'train_steps_per_second': 0.698, 'total_flos': 2.1893532827099136e+17, 'train_loss': 0.05600325330829857, 'epoch': 5.0})

In [30]:
save_directory = "./final_model"

model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

('./final_model\\tokenizer_config.json',
 './final_model\\special_tokens_map.json',
 './final_model\\vocab.txt',
 './final_model\\added_tokens.json',
 './final_model\\tokenizer.json')

In [31]:
encoding = tokenizer(
    sample_text,
    return_tensors="pt",
    truncation=True,
    padding=True,
    max_length=128
)

encoding = encoding.to(device)

model.eval()
with torch.no_grad():
    outputs = model(**encoding)

predictions = outputs.logits.argmax(dim=-1).squeeze().tolist()
tokens = tokenizer.convert_ids_to_tokens(encoding["input_ids"].squeeze())
decoded_preds = [id2label[p] for p in predictions]

for token, label in zip(tokens, decoded_preds):
    print(f"{token:15} -> {label}")

[CLS]           -> O
Hello           -> O
,               -> O
my              -> O
name            -> O
is              -> O
Sarah           -> B-FIRSTNAME
Johnson         -> B-LASTNAME
.               -> O
I               -> O
live            -> O
at              -> O
42              -> B-BUILDINGNUMBER
Maple           -> B-STREET
Street          -> I-STREET
,               -> O
New             -> B-STATE
York            -> I-STATE
,               -> O
and             -> O
my              -> O
email           -> O
is              -> O
sa              -> B-EMAIL
##rah           -> I-EMAIL
.               -> I-EMAIL
j               -> I-EMAIL
@               -> I-EMAIL
g               -> I-EMAIL
##mail          -> I-EMAIL
.               -> I-EMAIL
com             -> I-EMAIL
.               -> O
My              -> O
phone           -> O
number          -> O
is              -> O
555             -> B-PHONENUMBER
-               -> I-PHONENUMBER
1234            -> I-PHONENUMBER
.         